In [1]:
import os
from dotenv import load_dotenv
from os import path
from pydub import AudioSegment


# Load environment variables from .azure/captainslog/.env
load_dotenv('.azure/captainslog/.env')
file = r"C:\Users\blaineperry\Downloads\output.wav"

In [2]:
import requests

# Get the environment variables
speech_key = os.getenv('AZURE_SPEECH_KEY')
speech_region = os.getenv('AZURE_SPEECH_REGION')
speech_endpoint = os.getenv('AZURE_SPEECH_ENDPOINT')

stt_api = 'api.cognitive.microsoft.us'

# Print the environment variables to verify they are loaded correctly
print(f"Speech Key: {speech_key}")
print(f"Speech Region: {speech_region}")
print(f"Speech Endpoint: {speech_endpoint}")

# Define the endpoint URL
url = f"https://{speech_region}.{stt_api}/speech/recognition/conversation/cognitiveservices/v1"

# Define the parameters
params = {
    'language': 'en-US',
    'format': 'detailed'
}

# Define the headers
headers = {
    'Ocp-Apim-Subscription-Key': speech_key,
    'Content-Type': 'audio/wav'
}

# Read the audio file
with open(file, 'rb') as audio_file:
    audio_data = audio_file.read()

# Make the POST request
response = requests.post(url, params=params, headers=headers, data=audio_data)

# Print the response
print(response.status_code)
print(response.json())

Speech Key: CO34dRMoGfb3pQ6xBo3pxUV2KUHPZce7Vgs5xkmWCuQDRtojCwZTJQQJ99BFAJhseKSIUj3iAAAYACOGOm6s
Speech Region: usgovvirginia
Speech Endpoint: https://cog-speechg5azofxckevgi.cognitiveservices.azure.us/
404
{'error': {'code': '404', 'message': 'Resource not found'}}


In [3]:
import azure.cognitiveservices.speech as speechsdk

usGovEndpoint = "wss://usgovvirginia.stt.speech.azure.us"
speech_config = speechsdk.SpeechConfig(endpoint=usGovEndpoint, subscription=speech_key)

In [ ]:
# def recognize_from_microphone():
#      # This example requires environment variables named "SPEECH_KEY" and "ENDPOINT"
#      # Replace with your own subscription key and endpoint, the endpoint is like : "https://YourServiceRegion.api.cognitive.microsoft.com"
#     speech_config.speech_recognition_language="en-US"

#     audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
#     speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

#     print("Speak into your microphone.")
#     speech_recognition_result = speech_recognizer.recognize_once_async().get()

#     if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
#         print("Recognized: {}".format(speech_recognition_result.text))
#     elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
#         print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
#     elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
#         cancellation_details = speech_recognition_result.cancellation_details
#         print("Speech Recognition canceled: {}".format(cancellation_details.reason))
#         if cancellation_details.reason == speechsdk.CancellationReason.Error:
#             print("Error details: {}".format(cancellation_details.error_details))
#             print("Did you set the speech resource key and endpoint values?")

# # recognize_from_microphone()

In [11]:
from pydub import AudioSegment
import tempfile

def split_audio(input_file):
    audio = AudioSegment.from_file(input_file)

    # Define the chunk length (e.g., 30 seconds)
    chunk_duration_ms = 30 * 1000 # in milliseconds

    # Calculate number of chunks
    num_chunks = (len(audio) // chunk_duration_ms)+1

    # Split the audio file into chunks
    chunks = []
    for i in range(num_chunks):
        start = i * chunk_duration_ms
        end = (i + 1) * chunk_duration_ms
        chunk = audio[start:end]
        chunks.append(chunk)
    return chunks

def transcribe(filename: str):
    # Split audio file into chunks
    audio_chunks = split_audio(filename)
    print("Number of audio chunks: {}".format(len(audio_chunks)))
    print(f"Transcribing {len(audio_chunks)} audio chunks from {filename}")
    print(f"Audio duration: {len(AudioSegment.from_file(filename)) / 1000} seconds")

    full_transcription = ""
    # Transcribe each chunk
    for chunk in audio_chunks:

        # Create a temporary file to store the audio chunk
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio_file:
            chunk.export(temp_audio_file.name, format="wav")

            audio_config = speechsdk.audio.AudioConfig(filename=temp_audio_file.name)
            speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
            
            transcription = speech_recognizer.recognize_once()
            print(f"Transcription for chunk: {transcription.json}")

        # return transcription
            if isinstance(transcription, dict):
                text = transcription['text']
            else:
                text = transcription.text
            print(text)
            full_transcription = full_transcription + text
            
        # Close and Delete the temporary audio file
        temp_audio_file.close()
        # os.unlink(temp_audio_file.name)
    return full_transcription

In [12]:
file = r"C:\Users\blaineperry\Downloads\output (1).wav"
# file = r"C:\Users\blaineperry\Downloads\MSFT_OSS Azure AI Services discussion [In-person]-20250306_125903-Meeting Recording.mp4"
# file = r"C:\Users\BLAINE~1\AppData\Local\Temp\tmpqt2h_ugu.wav"
# file = r"C:\Users\blaineperry\Downloads\test.mp3"
# transcribe(file)
transcription = transcribe(file)
transcription

Number of audio chunks: 1
Transcribing 1 audio chunks from C:\Users\blaineperry\Downloads\output (1).wav
Audio duration: 1.238 seconds
Transcription for chunk: {"Id":"d3d91a5a6a8745fcb6f9ee8d76589cfb","RecognitionStatus":"Success","DisplayText":"Download me.","Offset":700000,"Duration":10900000,"Channel":0}
Download me.


'Download me.'

In [7]:
# import required modules


# assign files
filename = r"C:\Users\blaineperry\Downloads\test.mp3"
filename = file
output_file = "result.wav"

# get the filetype from the input file text
file_extension = path.splitext(filename)[1].lower()[1:]
print(f"Input file: {filename}, File extension: {file_extension}")
# convert mp3 file to wav file
sound = AudioSegment.from_file(filename, file_extension)
# Split audio into 30-second chunks
chunk_duration_ms = 30 * 1000  # 30 seconds in milliseconds
chunks = []

for i in range(0, len(sound), chunk_duration_ms):
    chunk = sound[i:i + chunk_duration_ms]
    chunks.append(chunk)

print(f"Created {len(chunks)} chunks of 30 seconds each")

Input file: C:\Users\blaineperry\Downloads\output (1).wav, File extension: wav
Created 1 chunks of 30 seconds each


In [8]:
# Read the file as a bytes IO object
with open(file, 'rb') as f:
    audio_bytes = f.read()

# Create a BytesIO object from the audio data
from io import BytesIO
audio_io = BytesIO(audio_bytes)

In [9]:
audio_io

In [10]:
import tempfile


# Create a named temporary file and write the BytesIO object to it
with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
    temp_file.write(audio_io.getvalue())
    temp_filename = temp_file.name

print(f"Audio data written to temporary file: {temp_filename}")


Audio data written to temporary file: C:\Users\BLAINE~1\AppData\Local\Temp\tmpguyl17ei.wav
